In [1]:
# Dependencies (Needs songephys environment)
import pandas as pd
import pickle as pk
import numpy as np
import h5py
#from soundsig.sound import BioSound 
import os, sys

/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Google drive helpers

In [32]:
import google_auth_oauthlib.flow
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
 
class Auth:
 
    def __init__(self, client_secret_filename, scopes):
        self.client_secret = client_secret_filename
        self.scopes = scopes
        self.flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(self.client_secret, self.scopes)
        self.flow.redirect_uri = 'http://localhost:8080/'
        self.creds = None
 
    def get_credentials(self):
        flow = InstalledAppFlow.from_client_secrets_file(self.client_secret, self.scopes)
        self.creds = flow.run_local_server(port=8080)
        return self.creds

 
# The scope you app will use. 
# (NEEDS to be among the enabled in your OAuth consent screen)
SCOPES = "https://www.googleapis.com/auth/drive.readonly"
CLIENT_SECRET_FILE = "/auto/zdrive/lthomas/Code/songephys/client_secrets.json"
 
credentials = Auth(client_secret_filename=CLIENT_SECRET_FILE, scopes=SCOPES).get_credentials()
 
drive_service = build('drive', 'v3', credentials=credentials)

def downloadfid(fid,outpath):
    cmd="curl -H \"Authorization: Bearer %s\" https://www.googleapis.com/drive/v3/files/%s?alt=media -o %s"%(credentials.token, fid,outpath)
    os.system(cmd)
def get_name_parent(fid):
    return drive_service.files().get(fileId=fid, fields='name, parents').execute()

def uploadfn(filename):
    media = MediaFileUpload(filename, mimetype="application/x-ustar", resumable=True)
    request = farm.animals().insert(media_body=media, body={'name': filename})
    response = None
    while response is None:
      status, response = request.next_chunk()
      if status:
        print("Uploaded %d%%." % int(status.progress() * 100))
    print( "Upload Complete!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=225544822063-j9rve5uf6io3ihiivrbn2rvorb779eo1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=JmheJlOv9URY1U5kih1v4UbuptXOlO&access_type=offline


## Redownload vocCuts and remake h5 files

In [42]:
results = drive_service.files().list(pageSize=10, q="name='vocCuts2020add.mat'",fields="files(id, name,parents)").execute()
downloadfid(results['files'][0]['id'],'/auto/zdrive/lthomas/FullVocBank_2020/vocCuts2020add.mat')

In [6]:
vocCutsContent = h5py.File('/auto/zdrive/lthomas/FullVocBank_2020/vocCuts2020.mat','r')

In [1]:
!ls /auto/zdrive/lthomas/FullVocBank_2020/

h5files		    vocCuts2020.mat	       vocParamTable.h5
vocCuts2020add.mat  vocParamTable_clusters.h5


### This code is copied from analyze_Calls_Julie.py/m

In [3]:
vocTypeCuts = np.array(vocCutsContent['vocTypeCuts']).squeeze()
vocTypeCutsStrs = ["".join([chr(x) for x in vocCutsContent[y]]) for y in vocTypeCuts]
fileNameCuts = np.array(vocCutsContent['soundNameCuts']).squeeze()
fileNameCutsStrs = ["".join([chr(x) for x in vocCutsContent[y]]) for y in fileNameCuts]

In [50]:
h5_outpath = "/auto/zdrive/lthomas/FullVocBank_2020/h5files"
if not os.path.exists(h5_outpath):
    os.makedirs(h5_outpath)
    
# set the order in which the call types should be displayed in confusion
# matrices
name_grp_plot = ['Be', 'LT', 'Tu', 'Th', 'Di', 'Ag', 'Wh', 'Ne', 'Te', 'DC', 'So'];

# Initialize some of the values
soundCutsAll = np.array(vocCutsContent["soundCutsAll"])
spectroCutsAll = np.array(vocCutsContent["spectroCutsAll"])
samprate = np.array(vocCutsContent["samprate"]).squeeze()
vocInds = np.array(vocCutsContent["indCutsAll"],dtype=int)
vocTypeCuts = np.array(vocCutsContent['vocTypeCuts']).squeeze()
vocTypeCutsStrs = ["".join([chr(x) for x in vocCutsContent[y]]) for y in vocTypeCuts]
birdNameCuts = np.array(vocCutsContent['birdNameCuts']).squeeze()
birdNameCutsStrs = ["".join([chr(x) for x in vocCutsContent[y]]) for y in birdNameCuts]
fileNameCuts = np.array(vocCutsContent['soundNameCuts']).squeeze()
fileNameCutsStrs = ["".join([chr(x) for x in vocCutsContent[y]]) for y in fileNameCuts]

fo = np.array(vocCutsContent["fo"]).squeeze()
to = np.array(vocCutsContent["to"]).squeeze()

In [51]:
# fix up some data
birdNameCutsStrs[birdNameCutsStrs =='HpiHpi4748'] = 'HPiHPi4748'
for i,bn in enumerate(birdNameCutsStrs):
    if bn == 'HpiHpi4748':
        birdNameCutsStrs[i] = 'HPiHPi4748'
for i,ct in enumerate(vocTypeCutsStrs):
    if ct == '-A':
        vocTypeCutsStrs[i] = 'Ag'
for i,fn in enumerate(fileNameCutsStrs):
    if fn[:18] == "GreOra1817_1108201":
        fileNameCutsStrs[i] = fn[:15]+fn[16:]


In [52]:
nsounds = np.shape(soundCutsAll)[1]      # Number of calls in the library

nf = max(np.shape(fo))   # Number of frequency slices in our spectrogram
nt = max(np.shape(to));      # Number of time slices in our spectrogram

# Loop through all sounds to extract temporal and spectral parameters
# can do this in parallel
def procSound(ind):
    soundIn  = soundCutsAll[:,ind]
    rms = np.std(soundIn[soundIn!=0]);
    soundSpect = np.reshape(spectroCutsAll[:,ind], (nt, nf))  # This is the spectrogram calculated in VocSectioningAmp
    
    # Create BioSound Object and store some values
    myBioSound = BioSound(soundWave = soundIn, fs = samprate, emitter=birdNameCutsStrs[ind], calltype = vocTypeCutsStrs[ind])
    myBioSound.spectro = soundSpect    # Log spectrogram
    myBioSound.to = to         # Time scale for spectrogram
    myBioSound.fo = fo         # Frequency scale for spectrogram
    myBioSound.rms = rms       # The rms
    
    # Calculate amplitude enveloppe
    myBioSound.ampenv()
       
    # Calculate the power spectrum
    myBioSound.spectrum(f_high=10000)
    
    # Calculate fundamental and related values
    myBioSound.fundest()
      
    # add a new attribute for the orig filename
    myBioSound.source_fn = np.string_(fileNameCutsStrs[ind])
    myBioSound.inds = vocInds[:,ind]
    # Save the results
    fname = '%s/%s_%s_%d.h5' % (h5_outpath,bytes(myBioSound.emitter).decode('utf-8'), bytes(myBioSound.type).decode('utf-8'), ind)
    myBioSound.saveh5(fname)

from multiprocessing import Pool
import tqdm
pool = Pool(8)
r = list(tqdm.tqdm(pool.imap(procSound, range(nsounds)), total=nsounds))


  1%|          | 57/8542 [02:02<4:29:50,  1.91s/it]/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/scipy/optimize/minpack.py:454: RuntimeWarning: Number of calls to function has reached maxfev = 2200.
  warnings.warn(errors[info][0], RuntimeWarning)

  1%|          | 84/8542 [02:59<6:20:26,  2.70s/it]/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/scipy/optimize/minpack.py:454: RuntimeWarning: Number of calls to function has reached maxfev = 2200.
  warnings.warn(errors[info][0], RuntimeWarning)

  1%|▏         | 107/8542 [03:48<6:20:34,  2.71s/it]/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/scipy/optimize/minpack.py:454: RuntimeWarning: Number of calls to function has reached maxfev = 2200.
  warnings.warn(errors[info][0], RuntimeWarning)

  1%|▏         | 112/8542 [04:10<8:28:26,  3.62s/it] /auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/scipy/optimize/minpack.py:454: RuntimeWarning: Number of cal

### Now to make vocParamTable, which will be used in further analysis.
This code is copied from Make_DataFrames_Calls_script1

In [53]:
import glob
import soundsig
import pandas,tqdm
from multiprocessing import Pool
import soundfile as sf


In [56]:
# Read the data
#h5_outpath = "/auto/zdrive/lthomas/FullVocBank_2020/h5files"
wav_folder = "/auto/fhome/julie/Documents/FullVocalizationBank/"
h5_files = glob.glob("%s/*"%h5_outpath)
store = '/auto/zdrive/lthomascd/FullVocBank_2020/vocParamTable.h5'


def proc_h5(fname):
    bsound = BioSound()
    bsound.readh5(fname)
    # removed due to lack of data:
    # sex, age
    # Added fname
    if bsound.fund.shape != (0,):
        fund = np.float(bsound.fund)
        cvfund = np.float(bsound.cvfund)
        maxfund = np.float(bsound.maxfund)
        minfund = np.float(bsound.minfund)
    else:
        fund = None 
        cvfund = None
        maxfund = None # added 
        minfund= None # added
    meanF1 = np.mean(bsound.F1[~np.isnan(bsound.F1)])
    meanF2 = np.mean(bsound.F2[~np.isnan(bsound.F2)])
    meanF3 = np.mean(bsound.F3[~np.isnan(bsound.F3)])

    bname = bytes(bsound.emitter).decode('utf-8')
    if bname == 'HpiHpi4748':
        bname = 'HPiHPi4748'
        print("OOP")
    ct = bytes(bsound.type).decode('utf-8')
    if ct == '-A':
        ct = 'Ag'
        print("OOP")
    fn = bytes(bsound.source_fn).decode('utf-8')
    if fn[:18] == "GreOra1817_1108201":
        fn = fn[:15]+fn[16:]
        print("OOP")
    if not os.path.exists("%s%s.wav"%(wav_folder,fn)):
        dur = 0.0
        sr = 0
        print("Missing %s%s.wav"%(wav_folder,fn))
    else:
        f = sf.SoundFile("%s%s.wav"%(wav_folder,fn))
        dur = len(f) / f.samplerate
        sr = f.samplerate
    return {"Bird": bname,
                     "calltype": ct, "fund": fund, 
                     "cvfund": cvfund, "maxfund": maxfund, "minfund": minfund,
                     "F1": meanF1, "F2": meanF2, "F3":meanF3,
                     "sal": np.float(bsound.meansal), "rms": np.float(bsound.rms), 
                     "maxAmp": np.float(bsound.maxAmp),
                     "meanS": np.float(bsound.meanspect), "stdS": np.float(bsound.stdspect),
                     "skewS": np.float(bsound.skewspect), "kurtS": np.float(bsound.kurtosisspect), 
                     "entS": np.float(bsound.entropyspect),
                     "q1": np.float(bsound.q1), "q2": np.float(bsound.q2), "q3": np.float(bsound.q3),                  
                     "meanT": np.float(bsound.meantime), "stdT": np.float(bsound.stdtime),
                     "skewT": np.float(bsound.skewtime), "kurtT": np.float(bsound.kurtosistime),
                     "entT": np.float(bsound.entropytime), "filename": fn, "duration": dur, "inds":bsound.inds,"samplerate":sr}
    
pool = Pool(8)
vocSelData = list(tqdm.tqdm_notebook(pool.imap(proc_h5, h5_files), total=len(h5_files)))                   
vocSelTable = pandas.DataFrame(vocSelData)
vocSelTable.to_hdf(store, 'callTable', mode = 'w')
pool.close()

Missing /auto/fhome/julie/Documents/FullVocalizationBank/GraLbl0457_110428-DC-25.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110607-WhineC-06.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BluRas07dd_110407-DC-02.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110607-WhineC-04.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110607-WhineC-02.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110429-DC-11.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110607-WhineC-05.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/GraLbl0457_110509-NestC-03.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/GraLbl0457_110606-WhineCop-02.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110429-DC-20.wav
Missing /auto/fhome/julie/Documents/FullVocalizationBank/BlaLbl8026_110607-WhineC-03.wav
Missing /auto/fhome/julie/Documents/

/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered

/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['Bird', 'calltype', 'filename', 'inds']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
